块（block）可以描述单个层、由多个层组成的组件或整个模型本身。 
使用块进行抽象的一个好处是可以将一些块组合成更大的组件， 这一过程通常是递归的

从编程的角度来看，块由类（class）表示。
它的任何子类都必须定义一个将其输入转换为输出的前向传播函数， 并且必须存储任何必需的参数。 注意，有些块不需要任何参数。 
最后，为了计算梯度，块必须具有反向传播函数。 在定义我们自己的块时，由于自动微分（在 2.5节 中引入） 提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。
除非我们实现一个新的运算符， 否则我们不必担心反向传播函数或参数初始化，系统将自动生成这些。

先回顾一下多层感知机(4.3节)的代码。 
下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层， 然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。


通过实例化nn.Sequential来构建我们的模型， 层的执行顺序是作为参数传递的。 
简而言之，nn.Sequential定义了一种特殊的Module， 即在PyTorch中表示一个块的类， 它维护了一个由Module组成的有序列表。 
注意，两个全连接层都是Linear类的实例， Linear类本身就是Module的子类。 另外，到目前为止，我们一直在通过net(X)调用我们的模型来获得模型的输出。 
这实际上是net.__call__(X)的简写。 这个前向传播函数非常简单： 它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0929,  0.0906,  0.1686,  0.1215,  0.0900, -0.1958, -0.0405, -0.1145,
          0.0307, -0.3643],
        [-0.0454,  0.1389,  0.1040,  0.0516,  0.1166, -0.1286,  0.0347, -0.1326,
         -0.0494, -0.1431]], grad_fn=<AddmmBackward0>)

每个块必须提供的基本功能

1.将输入数据作为其前向传播函数的参数。
2.通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
3.计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
4.存储和访问前向传播计算所需的参数。
5.根据需要初始化模型参数。

下面实现的类中包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。 注意，下面的MLP类继承了表示块的类。 我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

nn.Moudle是PyTorch中表示块的基础类，所有神经网络组件都应当继承这个类

编写 self.hidden = nn.Linear(20, 256) 时，实际发生了以下过程：
    创建实例：实例化一个 nn.Linear 层对象，输入维度为 20，输出维度为 256
    自动注册参数：由于 nn.Linear 也是 nn.Module 的子类，它内部包含权重和偏置参数
    子模块关联：将这个层对象赋值给 self.hidden，PyTorch 自动将其注册为当前模块的子模块

这是PyTorch nn.Moudle的一个重要特性，当你将一个 nn.Module 对象赋值给当前模块的属性时，PyTorch 会自动：
    将其注册为子模块（submodule）
    递归注册其所有参数
    在调用方法如 .parameters()、.to(device) 或 .train()/.eval() 时，这些操作会自动传播到所有子模块

子模块允许将复杂的神经网络分解成多个逻辑单元
每个子模块可以包含子集的参数和更小的子模块，形成层次结构

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__() # 调用MLP的父类Module的构造函数来执行必要的初始化。

        # 可以自动注册为模型的子模块和参数
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

尝试使用这个这个函数

In [3]:
net = MLP()
net(X)

tensor([[-0.2636, -0.0807,  0.1993,  0.2480,  0.1258, -0.0221,  0.1892,  0.0975,
         -0.0632, -0.0909],
        [-0.2110, -0.0437,  0.1833,  0.3855,  0.0157,  0.0157,  0.2237,  0.0154,
         -0.0104,  0.0410]], grad_fn=<AddmmBackward0>)

块的一个主要优点是它的多功能性。 我们可以子类化块以创建层（如全连接层的类）、 整个模型（如上面的MLP类）或具有中等复杂度的各种组件。

顺序块

Sequential的设计是为了把其他模块串起来。 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：
    一种将块逐个追加到列表中的函数；
    一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。
    下面的MySequential类提供了与默认Sequential类相同的功能。

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):  # args是一个可变参数列表，用于接受任意数量的模块
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            
            # python定义类时不需要提前声明成员变量或者属性
            # 变量_modules中。_module的类型是OrderedDict(有序字典)
            self._modules[str(idx)] = module 
            # 将模块添加到有序字典中，键为字符串索引，用于跟踪所有注册的子模块

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们，从而实现顺序块
        for block in self._modules.values():
            X = block(X)
        return X

当MySequential的前向传播函数被调用时， 每个添加的块都按照它们被添加的顺序执行。 现在可以使用我们的MySequential类重新实现多层感知机。

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0566,  0.0272,  0.0300,  0.2010, -0.1783,  0.1892, -0.0607,  0.2584,
         -0.0353,  0.0062],
        [ 0.0645,  0.0600, -0.0169,  0.1683, -0.1530,  0.1708, -0.0457,  0.2143,
         -0.2358, -0.0016]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

Sequential类使模型构造变得简单，允许我们组合新的架构，而不必定义自己的类。 
然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。 
例如，我们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算， 、而不是简单地依赖预定义的神经网络层。

到目前为止， 我们网络中的所有操作都对网络的激活值及网络的参数起作用。 然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项， 我们称之为常数参数（constant parameter）。 例如，我们需要一个计算函数 
$f(X,W)=c \cdot W^TX$的层， 其中X是输入，W是参数， c是某个在优化过程中没有更新的指定常量。 
因此我们实现了一个FixedHiddenMLP类，如下所示：

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变

        # 通过设置requires_grad=False来创建一个常量参数
        # 这意味着这个权重在反向传播中不会被更新，因此在训练过程中保持不变，是一个常数参数
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X) # 将输入X通过线性层进行变换

        # 使用创建的常量参数以及relu和mm函数
        # X乘上固定的随机权重矩阵，并加上常数偏置1，也就是c * W^T X这一过程
        # 然后使用ReLU激活函数处理结果
        X = F.relu(torch.mm(X, self.rand_weight) + 1)

        # 复用全连接层，因为使用的仍然是网络中原来的那个线性层。这相当于两个全连接层共享参数
        # 这就不是顺序块的执行逻辑
        X = self.linear(X) 

        # 控制流
        # 通过迭代触发确保输出的数值稳定性
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

使用该网络

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.2006, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。

嵌套了一个Sqquential和一个线性层

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1758, grad_fn=<SumBackward0>)